In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
#!pip install raceplotly
#from raceplotly.plots import barplot

In [ ]:
players = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/players.csv')
seasons = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/seasons.csv')
awards = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/awards.csv')
teams = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/teams.csv')
train = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/train.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train

I am using an unnested dataset created by @Ml_Bear link of the original [kernel](https://www.kaggle.com/naotaka1128/creating-unnested-dataset) do check it out

In [ ]:
train_next_day = pd.read_pickle('../input/mlb-unnested/train_nextDayPlayerEngagement.pickle')
train_next_day.engagementMetricsDate = train_next_day.engagementMetricsDate.astype('datetime64')

In [ ]:
train_next_day

In [ ]:
train_next_day.head()

In [ ]:
unique_date = list(train_next_day.engagementMetricsDate.unique())
target1_lis = []
target2_lis = []
target3_lis = []
target4_lis = []
for i in unique_date:
    df = train_next_day[train_next_day['engagementMetricsDate']==i]
    target1 = (df['target1'].sum())/len(df)
    target2 = (df['target2'].sum())/len(df)
    target3 = (df['target3'].sum())/len(df)
    target4 = (df['target4'].sum())/len(df)
    target1_lis.append(target1)
    target2_lis.append(target2)
    target3_lis.append(target3)
    target4_lis.append(target4)

In [ ]:
px.line(x=unique_date,y=target1_lis,title='target 1 over time')

In [ ]:
px.line(x=unique_date,y=target2_lis,title='target 2 over time')

In [ ]:
px.line(x=unique_date,y=target3_lis,title='target 3 over time')

In [ ]:
px.line(x=unique_date,y=target4_lis,title='target 4 over time')

In [ ]:
team_twitter = pd.read_pickle('../input/mlb-unnested/train_teamTwitterFollowers.pickle')
team_twitter

In [ ]:
#my_raceplot = barplot(team_twitter,  item_column='teamName', value_column='numberOfFollowers', time_column='date')
#my_raceplot.plot(item_label = 'team name', value_label = 'number of followers', frame_duration = 800)

In [ ]:
standings = pd.read_pickle('../input/mlb-unnested/train_standings.pickle')
transactions = pd.read_pickle('../input/mlb-unnested/train_transactions.pickle')

In [ ]:
#my_raceplot = barplot(standings,  item_column='teamName',value_column='wins', time_column='dailyDataDate',top_entries=10)
#my_raceplot.plot(item_label = 'team name', value_label = 'current wins', frame_duration = 800)

In [ ]:
transactions = transactions.dropna()

In [ ]:
transactions

In [ ]:
most_transaction_team=transactions.groupby('fromTeamName')['toTeamName'].count().reset_index(name='Count').sort_values('Count',ascending=False)
px.bar(most_transaction_team.head(10),x='fromTeamName',y='Count')

In [ ]:
most_transaction_player=transactions.groupby('playerName')['toTeamName'].count().reset_index(name='Count').sort_values('Count',ascending=False)
px.bar(most_transaction_player.head(10),x='playerName',y='Count')

In [ ]:
players.head()

In [ ]:
players.info()

In [ ]:
px.histogram(players,x='birthCountry',color='birthCountry')

In [ ]:
px.scatter(players,x='weight',y='heightInches')

In [ ]:
playerid = list(awards['playerId'])
award_count = []
for i in playerid:
    award_count.append(len(awards[awards['playerId']==i]))
award_count = pd.DataFrame({"playerId":playerid,"award_count":award_count})
players = pd.merge(players,award_count,on='playerId')

In [ ]:
postition_list = list(players['primaryPositionName'].unique())
award_count_sum = []
for i in postition_list:
    award_count_sum.append(players[players['primaryPositionName']==i]['award_count'].sum()) 

In [ ]:
px.bar(x=postition_list,y=award_count_sum)

In [ ]:
most_award = awards['playerId'].mode()
print(f"playerID: {most_award.values}")
awards[awards['playerId'] == 405395]

In [ ]:
px.histogram(awards,y='awardName',category_orders=awards['awardName'])

In [ ]:
sample_preiction = pd.read_pickle('../input/mlb-unnested/example_sample_submission.pickle')
example_test_games = pd.read_pickle('../input/mlb-unnested/example_test_games.pickle')
train_games = pd.read_pickle('../input/mlb-unnested/train_games.pickle')
train_next_day = pd.read_pickle('../input/mlb-unnested/train_nextDayPlayerEngagement.pickle')
train_next_day['year'] = pd.DatetimeIndex(train_next_day['engagementMetricsDate']).year

In [ ]:
year = train_next_day.year.unique()
weight=[0.05,0.05,0.1,0.8]#Experiment here with different values

In [ ]:
def weighted_median(df, val, weight):
    df_sorted = df.sort_values(val)
    cumsum = df_sorted[weight].cumsum()
    cutoff = df_sorted[weight].sum() / 2.
    return df_sorted[cumsum >= cutoff][val].iloc[0]

In [ ]:
def preprocess(df,weight,year):
    for i in range(len(year)):
        df.loc[df['year']==year[i],'weight'] = weight[i]
preprocess(train_next_day,weight,year)

In [ ]:
train_next_day.head()

In [ ]:
playerId = train_next_day['playerId'].unique()
for i in playerId:
    df=train_next_day[train_next_day['playerId']==i]
    wm1 = weighted_median(df,'target1','weight')
    wm2 = weighted_median(df,'target2','weight')
    wm3 = weighted_median(df,'target3','weight')
    wm4 = weighted_median(df,'target4','weight')
    train_next_day.loc[train_next_day['playerId']==i,'target1'] = wm1
    train_next_day.loc[train_next_day['playerId']==i,'target2'] = wm2
    train_next_day.loc[train_next_day['playerId']==i,'target3'] = wm3
    train_next_day.loc[train_next_day['playerId']==i,'target4'] = wm4

In [ ]:
train_mean = train_next_day.groupby(["playerId"])[["target1","target2","target3","target4"]].median().reset_index()

In [ ]:
train_mean.head()

In [ ]:
def process_pred(df):
    df["playerId"] = df["date_playerId"].apply(lambda x: int( x.split("_")[1] ) )
    df.drop(["target1","target2","target3","target4"], axis=1, inplace=True)
    df = df.merge(train_mean, on="playerId", how="left")
    df.drop("playerId", axis=1, inplace=True)
    df = df.fillna(0.)
    return df

In [ ]:
import mlb
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df = process_pred(sample_prediction_df)
    env.predict(sample_prediction_df)

In [ ]:
sample_prediction_df.head()